<a href="https://colab.research.google.com/github/zhuzihan728/metal-binding-site-prediction/blob/main/cluster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install biopython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.1 MB 7.5 MB/s 


In [2]:
!tar -xvf /content/drive/MyDrive/miniconda -C /root

流式输出内容被截断，只能显示最后 5000 行内容。
miniconda/lib/python3.7/site-packages/conda_env/cli/__pycache__/main_config.cpython-37.pyc
miniconda/lib/python3.7/site-packages/conda_env/cli/main_config.py
miniconda/lib/python3.7/site-packages/conda_env/cli/main.py
miniconda/lib/python3.7/site-packages/conda_env/cli/main_vars.py
miniconda/lib/python3.7/site-packages/conda_env/exceptions.py
miniconda/lib/python3.7/site-packages/conda_env/installers/
miniconda/lib/python3.7/site-packages/conda_env/installers/__init__.py
miniconda/lib/python3.7/site-packages/conda_env/installers/conda.py
miniconda/lib/python3.7/site-packages/conda_env/installers/base.py
miniconda/lib/python3.7/site-packages/conda_env/installers/__pycache__/
miniconda/lib/python3.7/site-packages/conda_env/installers/__pycache__/pip.cpython-37.pyc
miniconda/lib/python3.7/site-packages/conda_env/installers/__pycache__/conda.cpython-37.pyc
miniconda/lib/python3.7/site-packages/conda_env/installers/__pycache__/__init__.cpython-37.pyc
miniconda/lib

In [3]:
 !tar -xvf /content/drive/MyDrive/clusters -C /content
 # !mmseqs easy-cluster /content/combined.fasta assembly_clustered tmp --cov-mode 5 -c 0.25 --min-seq-id 0.4

assembly_clustered_all_seqs.fasta
assembly_clustered_rep_seq.fasta
assembly_clustered_cluster.tsv


In [4]:
!tar -xvf /content/drive/MyDrive/uniprot_dataset.tar -C /content

ChEBI-IDs_for_metal_binding.tsv
POS_TRAIN_FULL.tsv
POS_TRAIN.tsv
NEG_TRAIN.fasta
POS_TRAIN.fasta
POS_TRAIN_FULL.fasta


In [5]:
!cat NEG_TRAIN.fasta	POS_TRAIN_FULL.fasta > combined.fasta

In [6]:
%alias activate $HOME/miniconda/bin/activate

In [7]:
%alias mmseqs $HOME/miniconda/pkgs/mmseqs2-14.7e284-pl5321hf1761c0_0/bin/mmseqs

In [8]:
activate tutorial

In [9]:
mmseqs

MMseqs2 (Many against Many sequence searching) is an open-source software suite for very fast, 
parallelized protein sequence searches and clustering of huge protein sequence data sets.

Please cite: M. Steinegger and J. Soding. MMseqs2 enables sensitive protein sequence searching for the analysis of massive data sets. Nature Biotechnology, doi:10.1038/nbt.3988 (2017).

MMseqs2 Version: 14.7e284
© Martin Steinegger (martin.steinegger@snu.ac.kr)

usage: mmseqs <command> [<args>]

Easy workflows for plain text input/output
  easy-search       	Sensitive homology search
  easy-cluster      	Slower, sensitive clustering
  easy-linclust     	Fast linear time cluster, less sensitive clustering
  easy-taxonomy     	Taxonomic classification
  easy-rbh          	Find reciprocal best hit

Main workflows for database input/output
  search            	Sensitive homology search
  map               	Map nearly identical sequences
  rbh               	Reciprocal best hit search
  linclust          	F

In [10]:
mmseqs easy-cluster

usage: mmseqs easy-cluster <i:fastaFile1[.gz|.bz2]> ... <i:fastaFileN[.gz|.bz2]> <o:clusterPrefix> <tmpDir> [options]
options:                               
 -c FLOAT                       List matches above this fraction of aligned (covered) residues (see --cov-mode) [0.800]
 --cov-mode INT                 0: coverage of query and target
                                1: coverage of target
                                2: coverage of query
                                3: target seq. length has to be at least x% of query length
                                4: query seq. length has to be at least x% of target length
                                5: short seq. needs to be at least x% of the other seq. length [0]
 --alignment-mode INT           How to compute the alignment:
                                0: automatic
                                1: only score and end_pos
                                2: also start_pos and cov
                                3: also seq.i

In [ ]:
#!cat /content/assembly_clustered_all_seqs.fasta

In [11]:
from Bio import SeqIO

In [12]:
print('negative data:', len(list(SeqIO.parse("NEG_TRAIN.fasta", "fasta"))))
print('positive data:', len(list(SeqIO.parse("POS_TRAIN_FULL.fasta", "fasta"))))
print('total:', len(list(SeqIO.parse("combined.fasta", "fasta"))))

negative data: 193133
positive data: 93894
total: 287027


In [13]:
ls = [] # a list of [accession, sequence]
for seq_record in SeqIO.parse("POS_TRAIN_FULL.fasta", "fasta"):
  ls.append([seq_record.id.split('|')[1], str(seq_record.seq)])

In [14]:
import pandas as pd

anno = pd.read_csv('POS_TRAIN_FULL.tsv', sep='\t')
anno

,Accession,Evidence,ChEBI-ID,Position
0,P0C6X8,ECO:0000255,CHEBI:29105,6190
1,P0C6X8,ECO:0000255,CHEBI:29105,6376
2,Q9RS27,ECO:0000255,CHEBI:29105,693
3,Q5HLY0,ECO:0000255,CHEBI:18420,97
4,Q9I2T1,ECO:0000255,CHEBI:29105,170
...,...,...,...,...
409984,C5JYZ5,ECO:0000255,CHEBI:60240,318
409985,C5JYZ5,ECO:0000255,CHEBI:60240,446
409986,A7FLX7,ECO:0000255,CHEBI:60240,8
409987,A7FLX7,ECO:0000255,CHEBI:60240,44


In [15]:
metal_count_df = anno['ChEBI-ID'].value_counts().to_frame().reset_index()
metal_count_df.columns = ['ChEBI-ID', 'count']
print(metal_count_df)

        ChEBI-ID   count
0    CHEBI:29105  133807
1    CHEBI:18420   89414
2    CHEBI:49883   51607
3    CHEBI:29108   42179
4    CHEBI:29035   22438
5    CHEBI:60240   18192
6    CHEBI:24875   17448
7   CHEBI:190135    9036
8    CHEBI:23378    7395
9    CHEBI:29103    6266
10   CHEBI:49786    3395
11   CHEBI:29101    2234
12   CHEBI:29034    1659
13   CHEBI:30408    1058
14   CHEBI:29036     934
15   CHEBI:29033     832
16   CHEBI:21137     510
17   CHEBI:49552     368
18   CHEBI:48775     361
19   CHEBI:48828     229
20   CHEBI:21143     176
21   CHEBI:25213     131
22   CHEBI:47739     106
23   CHEBI:16793      90
24  CHEBI:177874      51
25   CHEBI:60400      44
26   CHEBI:49415      16
27   CHEBI:60504       7
28   CHEBI:49713       6


In [16]:
print('number of prot seqs:', len(list(SeqIO.parse("combined.fasta", "fasta"))))
# 287027

number of prot seqs: 287027


In [17]:
print('number of clusters:', len(list(SeqIO.parse("assembly_clustered_rep_seq.fasta", "fasta"))))
# 38717

number of clusters: 38717


In [18]:
clusters = pd.read_csv('assembly_clustered_cluster.tsv', sep='\t', header=None)
clusters.columns = ['Rep', 'Accession']
clusters

,Rep,Accession
0,P0C8W0,P0C8W0
1,P0DP77,P0DP77
2,P10208,P10208
3,P19063,P19063
4,P33613,P33613
...,...,...
287022,Q80XR2,Q64566
287023,Q80XR2,A7L9Z8
287024,Q80XR2,O75185
287025,Q80XR2,Q8R4C1


In [25]:
print('number of reps: %d' % len(clusters['Rep'].unique())) # check if #rep == #cluster

number of reps: 38717


In [28]:
print('number of clustered seqs: %d' % len(clusters)) # check if #clustered seqs == #seqs
# 287027

number of clustered seqs: 287027


In [29]:
cluster_label = pd.merge(clusters, anno, on = 'Accession', how = "outer")
cluster_label.dropna(axis=0, how='any', inplace=True)
cluster_label

,Rep,Accession,Evidence,ChEBI-ID,Position
629,A7MQJ1,Q1RGX4,ECO:0000255,CHEBI:49883,86.0
630,A7MQJ1,Q1RGX4,ECO:0000255,CHEBI:49883,112.0
631,A7MQJ1,Q1RGX4,ECO:0000255,CHEBI:49883,119.0
632,A7MQJ1,Q1RGX4,ECO:0000255,CHEBI:49883,325.0
633,A7MQJ1,Q1RGX4,ECO:0000255,CHEBI:49883,91.0
...,...,...,...,...,...
603117,Q80XR2,Q8R4C1,ECO:0000250,CHEBI:29108,770.0
603118,Q81RQ4,Q81RQ4,ECO:0000269,CHEBI:29035,253.0
603119,Q81RQ4,Q81RQ4,ECO:0000269,CHEBI:29035,198.0
603120,Q81RQ4,Q81RQ4,ECO:0000269,CHEBI:29035,172.0


In [30]:
rep_metal = cluster_label[['Rep', 'ChEBI-ID']].value_counts().to_frame().reset_index()
rep_metal.columns = ['Rep', 'ChEBI-ID', 'count']
rep_metal

,Rep,ChEBI-ID,count
0,Q8HUH0,CHEBI:29105,5912
1,P05934,CHEBI:29108,5052
2,B9MQ24,CHEBI:49883,4150
3,Q72GN6,CHEBI:29105,3131
4,Q88VW2,CHEBI:18420,2732
...,...,...,...
7282,P33453,CHEBI:18420,1
7283,P33182,CHEBI:18420,1
7284,Q9XGM8,CHEBI:29035,1
7285,Q8EVH2,CHEBI:18420,1


In [31]:
'P33453' not in rep_metal['Rep'].unique()

False

In [41]:
rep_Allmetal = cluster_label['Rep'].value_counts().to_frame().reset_index()
rep_Allmetal.columns = ['Rep', 'count']
reps = list(rep_Allmetal['Rep'].unique())
import random
random.seed(42)
random.shuffle(reps)
reps[:10]


['Q9UW26',
 'Q55506',
 'Q8EVF5',
 'Q54Q80',
 'Q8A1A0',
 'Q59039',
 'O15743',
 'A2BGT0',
 'O26246',
 'F4JJ23']

In [42]:
seq_metal = cluster_label[['Accession', 'ChEBI-ID']].value_counts().to_frame().reset_index()
seq_metal.columns = ['Accession', 'ChEBI-ID', 'count']
seq_metal

,Accession,ChEBI-ID,count
0,O31526,CHEBI:29108,88
1,E0VIU9,CHEBI:29105,84
2,O31527,CHEBI:29108,81
3,Q9Y4X5,CHEBI:29105,72
4,Q7KTX7,CHEBI:29105,63
...,...,...,...
102525,A9NGK2,CHEBI:18420,1
102526,Q03216,CHEBI:18420,1
102527,Q03215,CHEBI:18420,1
102528,A7NEB4,CHEBI:29103,1


In [44]:
seq_Allmetal = cluster_label['Accession'].value_counts().to_frame().reset_index()
seq_Allmetal.columns = ['Accession', 'count']
seqs = list(seq_Allmetal['Accession'].unique())
random.shuffle(seqs)
seqs[:10]

['P80164',
 'Q89DJ1',
 'Q817F3',
 'P25821',
 'B5BJR5',
 'Q747X9',
 'O13220',
 'P0ADF7',
 'Q9JID2',
 'B6EKA0']

In [48]:
print('number of clusters containing metalloprotein: %d' % len(reps))

number of clusters containing metalloprotein: 6590


In [50]:
non_metal_reps = []
for i in clusters['Rep'].unique():
  if i not in reps:
    non_metal_reps.append(i)
print('number of clusters not containing metalloprotein: %d' % len(non_metal_reps))

number of clusters not containing metalloprotein: 32127


In [51]:
import numpy as np

In [52]:
import math

metal_count_test = metal_count_df.copy()
metal_count_test['count'] = metal_count_test['count']*0.1//1+1

metal_test = set()
test = []
trainval = []
for i in reps:
  print(f'----- decide for cluster {i} -----')

  temp = rep_metal[rep_metal['Rep'] == i].reset_index()
  print(f'cluster {i} has annotations:\n{temp}')
  flag = 1
  for m in temp['ChEBI-ID']:
    num = temp[temp['ChEBI-ID']==m].iloc[0,3]
    cur = metal_count_test.loc[metal_count_test['ChEBI-ID']==m].iloc[0,1]
    max_num = metal_count_df.loc[metal_count_df['ChEBI-ID']==m].iloc[0,1]
    if cur + 10 < num or num == max_num:
      print(f'metal {m} exceed maximum, cluster {i} -> trainval')
      trainval.append(i)
      flag = 0
      break
  if flag:
    test.append(i)
    print(f'cluster {i} -> test')
    for n in temp['ChEBI-ID']: 
        print(f"the cluster has metal {n}, number {temp[temp['ChEBI-ID']==n].iloc[0,3]}")
        print(metal_count_test)
        metal_count_test.loc[metal_count_test['ChEBI-ID']==n, 'count'] -= temp[temp['ChEBI-ID']==n].iloc[0,3]
        print(metal_count_test)
        metal_test.add(n)
        print(f"update metal presented in test set: {metal_test}")
  print('\n\n')

randnums = np.random.randint(0, len(non_metal_reps), int(0.1*len(non_metal_reps)))
for i, v in enumerate(non_metal_reps):
  if i in randnums:
    test.append(v)
  else:
    trainval.append(v)
print(test)
print(trainval)



流式输出内容被截断，只能显示最后 5000 行内容。
   index     Rep     ChEBI-ID  count
0   3681  P0AFC2  CHEBI:18420      9
metal CHEBI:18420 exceed maximum, cluster P0AFC2 -> trainval



----- decide for cluster P07717 -----
cluster P07717 has annotations:
   index     Rep     ChEBI-ID  count
0   7034  P07717  CHEBI:60240      2
metal CHEBI:60240 exceed maximum, cluster P07717 -> trainval



----- decide for cluster Q49611 -----
cluster Q49611 has annotations:
   index     Rep     ChEBI-ID  count
0   2693  Q49611  CHEBI:49883     16
metal CHEBI:49883 exceed maximum, cluster Q49611 -> trainval



----- decide for cluster O34543 -----
cluster O34543 has annotations:
   index     Rep     ChEBI-ID  count
0   4694  O34543  CHEBI:24875      6
metal CHEBI:24875 exceed maximum, cluster O34543 -> trainval



----- decide for cluster Q54PA0 -----
cluster Q54PA0 has annotations:
   index     Rep     ChEBI-ID  count
0   2651  Q54PA0  CHEBI:29105     16
metal CHEBI:29105 exceed maximum, cluster Q54PA0 -> trainval



---

In [53]:
print(len(test))

3865


In [54]:
print(len(trainval))

34852


In [57]:
list(set(test) & set(trainval)) # check no intersection

[]

In [71]:
# for i in cluster_label[cluster_label['Rep'] == 'Q8HUH0']['ChEBI-ID']:
#       print(i)
def check_Allmetal(reps):
  stat = {}
  metals = metal_count_df['ChEBI-ID'].unique()
  metal_dic = dict(zip(metals, [0 for i in range(len(metals))]))
  for rep in reps:
    for i, row in rep_metal[rep_metal['Rep'] == rep].iterrows():
      metal_dic[row['ChEBI-ID']] += row['count']
  for i in metal_dic:
    per = metal_dic[i] / int(metal_count_df[metal_count_df['ChEBI-ID'] == i]['count'])
    stat[i] = [metal_dic[i], per]
    print(f'metal: {i}, num: {metal_dic[i]}, percentage: {per}')
  return stat

def check_metal(seqs, metal):
  cnt = 0
  temp = cluster_label.loc[cluster_label['Accession'].isin(seqs)]
  temp1 = temp['ChEBI-ID'].value_counts().to_frame().reset_index()
  row = temp1[temp1['index'] == metal]['ChEBI-ID']
  cnt = 0 if len(row) == 0 else int(row)
  per = cnt / int(metal_count_df[metal_count_df['ChEBI-ID'] == metal]['count'])
  return per


In [72]:
init_stat = check_Allmetal(test)

metal: CHEBI:29105, num: 13391, percentage: 0.10007697654083868
metal: CHEBI:18420, num: 8952, percentage: 0.10011854966783724
metal: CHEBI:49883, num: 5171, percentage: 0.10019958532757184
metal: CHEBI:29108, num: 4228, percentage: 0.10023945565328718
metal: CHEBI:29035, num: 2254, percentage: 0.10045458597022908
metal: CHEBI:60240, num: 1830, percentage: 0.10059366754617415
metal: CHEBI:24875, num: 1755, percentage: 0.10058459422283356
metal: CHEBI:190135, num: 914, percentage: 0.10115095174856131
metal: CHEBI:23378, num: 750, percentage: 0.10141987829614604
metal: CHEBI:29103, num: 637, percentage: 0.1016597510373444
metal: CHEBI:49786, num: 350, percentage: 0.10309278350515463
metal: CHEBI:29101, num: 232, percentage: 0.10384959713518353
metal: CHEBI:29034, num: 110, percentage: 0.06630500301386377
metal: CHEBI:30408, num: 54, percentage: 0.05103969754253308
metal: CHEBI:29036, num: 4, percentage: 0.004282655246252677
metal: CHEBI:29033, num: 86, percentage: 0.10336538461538461
met

In [60]:
test_seqs = []
trainval_seqs = []
test_seqs = list(clusters.loc[clusters['Rep'].isin(test)]['Accession'])
trainval_seqs = list(clusters.loc[clusters['Rep'].isin(trainval)]['Accession'])
# for i in test:
#   for ind, seq in clusters[clusters['Rep'] == i].iterrows():
#     test_seqs.append(seq['Accession'])

# for i in trainval:
#   for ind, seq in clusters[clusters['Rep'] == i].iterrows():
#     trainval_seqs.append(seq['Accession'])
print(test_seqs)
print(trainval_seqs)


['P0C8W0', 'G5EFN6', 'Q7JGM0', 'O18794', 'Q7JGL8', 'P61263', 'P60022', 'Q7JGM1', 'P61262', 'Q7JGL9', 'Q95J22', 'Q95M68', 'P60023', 'Q7JGM2', 'Q95M66', 'Q95M67', 'P61261', 'Q95J24', 'Q95J18', 'Q7TNV7', 'O89117', 'P56386', 'Q8NFR7', 'P52067', 'P57193', 'Q89AY4', 'Q8KA28', 'P0A953', 'P0A954', 'Q02K94', 'P43710', 'Q09768', 'Q54PC2', 'Q20117', 'P32477', 'Q9HF78', 'Q9W3K5', 'Q44532', 'Q9L6V3', 'Q49396', 'Q4FNQ1', 'C8VI80', 'Q0UK52', 'A0A0C6DRT7', 'P24668', 'Q6AY20', 'P20645', 'P11456', 'P69607', 'P12649', 'Q910E2', 'Q91QS9', 'P69606', 'P11222', 'P69603', 'P69601', 'P69602', 'Q9J3N8', 'P69604', 'P05136', 'P69605', 'P77998', 'P28615', 'P08623', 'P0A172', 'P0A171', 'A0A0H2ZGR9', 'P49988', 'Q9S0L2', 'P06223', 'P24255', 'P26979', 'P24695', 'P33983', 'Q09272', 'P75330', 'Q49417', 'Q20411', 'Q29RH7', 'Q5R8B5', 'Q8BKT6', 'O95859', 'Q569A2', 'Q5ZIF5', 'Q5R456', 'Q6DC39', 'Q5PQ09', 'Q9BSY9', 'Q9D291', 'Q5ZIV7', 'A3QRX8', 'Q5XIT6', 'H2KZK4', 'Q8S8S1', 'P40332', 'P90893', 'Q03327', 'Q21489', 'Q2QL90', '

In [93]:
cnt = len(clusters.loc[clusters['Rep'].isin(test_seqs)])

cnt / len(clusters)

0.09532552686681044

In [61]:
list(set(test_seqs) & set(trainval_seqs)) # check no intersection

[]

In [79]:
# check if the two sets contain all data
len(test_seqs) + len(trainval_seqs)

287027

In [77]:
# some test code
b = cluster_label.loc[cluster_label['Accession'].isin(test_seqs)]
temp1 = b['ChEBI-ID'].value_counts().to_frame().reset_index()
int(temp1[temp1['index'] == 'CHEBI:49552']['ChEBI-ID'])

44

In [78]:
check_metal(test_seqs, 'CHEBI:21143') # check correctness: 0.011363636363636364

0.011363636363636364

In [ ]:
seq_metal.loc[(seq_metal['Accession'] == 'O31526') & (seq_metal['ChEBI-ID'] == 'CHEBI:29108')]['count'].iloc[0]

88

In [85]:
new_test_seqs = test_seqs.copy()
new_trainval_seqs = trainval_seqs.copy()


for i, v in init_stat.items():
  print(f'check for {i}')
  print(f'initial number is {v[0]}, percentage is {v[1]}')
  if v[1] < 0.099:
    for seq in seqs:
      if seq in new_trainval_seqs:
        if ((seq_metal['Accession'] == seq) & (seq_metal['ChEBI-ID'] == i)).any():
          new_trainval_seqs.remove(seq)
          new_test_seqs.append(seq)
          print(f'{seq} -> test')
          new_per = check_metal(new_test_seqs, i)
          print(f'updated percentage is {new_per}')
          if new_per >= 0.099:
            break

check for CHEBI:29105
initial number is 13391, percentage is 0.10007697654083868
check for CHEBI:18420
initial number is 8952, percentage is 0.10011854966783724
check for CHEBI:49883
initial number is 5171, percentage is 0.10019958532757184
check for CHEBI:29108
initial number is 4228, percentage is 0.10023945565328718
check for CHEBI:29035
initial number is 2254, percentage is 0.10045458597022908
check for CHEBI:60240
initial number is 1830, percentage is 0.10059366754617415
check for CHEBI:24875
initial number is 1755, percentage is 0.10058459422283356
check for CHEBI:190135
initial number is 914, percentage is 0.10115095174856131
check for CHEBI:23378
initial number is 750, percentage is 0.10141987829614604
check for CHEBI:29103
initial number is 637, percentage is 0.1016597510373444
check for CHEBI:49786
initial number is 350, percentage is 0.10309278350515463
check for CHEBI:29101
initial number is 232, percentage is 0.10384959713518353
check for CHEBI:29034
initial number is 110,

In [86]:
list(set(new_test_seqs) & set(new_trainval_seqs)) # check no intersection

[]

In [90]:
for metal in metal_count_df['ChEBI-ID'].unique():
  per = check_metal(new_test_seqs, metal) 
  num = int(metal_count_df[metal_count_df['ChEBI-ID'] == metal]['count'])
  print(f'{metal}| num: {int(num*per)} per: {per}')

CHEBI:29105| num: 13443 per: 0.10046559597031546
CHEBI:18420| num: 8960 per: 0.10020802111526159
CHEBI:49883| num: 5195 per: 0.1006646385180305
CHEBI:29108| num: 4228 per: 0.10023945565328718
CHEBI:29035| num: 2258 per: 0.10063285497816205
CHEBI:60240| num: 1830 per: 0.10059366754617415
CHEBI:24875| num: 1755 per: 0.10058459422283356
CHEBI:190135| num: 914 per: 0.10115095174856131
CHEBI:23378| num: 750 per: 0.10141987829614604
CHEBI:29103| num: 637 per: 0.1016597510373444
CHEBI:49786| num: 350 per: 0.10309278350515463
CHEBI:29101| num: 232 per: 0.10384959713518353
CHEBI:29034| num: 166 per: 0.10006027727546715
CHEBI:30408| num: 108 per: 0.10207939508506617
CHEBI:29036| num: 99 per: 0.10599571734475374
CHEBI:29033| num: 86 per: 0.10336538461538461
CHEBI:21137| num: 61 per: 0.11960784313725491
CHEBI:49552| num: 44 per: 0.11956521739130435
CHEBI:48775| num: 49 per: 0.13573407202216067
CHEBI:48828| num: 32 per: 0.13973799126637554
CHEBI:21143| num: 20 per: 0.11363636363636363
CHEBI:25213| 

In [95]:
cnt = len(clusters.loc[clusters['Rep'].isin(new_test_seqs)])

cnt / len(clusters)

0.09534643082358106

In [92]:
with open('/content/drive/MyDrive/test.txt', 'r+') as f:
  f.write(str(new_test_seqs))

with open('/content/drive/MyDrive/train.txt', 'r+') as f:
  f.write(str(new_trainval_seqs))